In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/3-2/"

/content/drive/MyDrive/Colab Notebooks/3-2


In [ ]:
#필요한 패키지 불러오기

import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

import yfinance as yf
yf.pdr_override()  # <-- Override to use yfinance when calling pandas_datareader

In [3]:
#바이오 기업들 Id 불러와서 리스트로 저장
import pandas as pd

df = pd.read_parquet('./inner_bio_companyId.parquet')
indexes = list(df['companyId'])

In [4]:
#바이오 마켓 데이터 불러오기
import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_market = pd.read_parquet('./market', filters=filters, columns=['companyId', 'pricingDate', 'priceClose', 'adjustmentFactor'])
df_market['companyId'] = list(map(int, df_market['companyId']))
df_market

,companyId,pricingDate,priceClose,adjustmentFactor
0,24153,2023-10-10,34.740000,1.0000000000
1,24153,2023-10-09,34.360000,1.0000000000
2,24153,2023-10-06,34.090000,1.0000000000
3,24153,2023-10-05,34.040000,1.0000000000
4,24153,2023-10-04,33.940000,1.0000000000
...,...,...,...,...
1798012,1844729817,2023-07-26,40.750000,1.0000000000
1798013,1844729817,2023-07-25,39.150000,1.0000000000
1798014,1844729817,2023-07-24,38.700000,1.0000000000
1798015,1844729817,2023-07-21,40.550000,1.0000000000


In [25]:
#바이오 예상치 데이터 불러오기
filters = [('companyId', 'in', indexes)]
df_estimates = pd.read_parquet('./estimates', filters=filters)

df_estimates['companyId'] = list(map(int, df_estimates['companyId']))
df_estimates['dataItemId'] = list(map(int, df_estimates['dataItemId']))
df_estimates['dataItemValue'] = list(map(float, df_estimates['dataItemValue']))
df_estimates['fiscalYear'] = list(map(int, df_estimates['fiscalYear']))

pd.options.mode.chained_assignment = None
df_est = df_estimates.copy()

#Estimates 데이터 단위 맞추는 작업

for i in range(len(df_est)):
  if df_est['estimateScaleName'][i] == 'Millions':
    df_est['dataItemValue'][i] = df_est['dataItemValue'][i] * 1_000_000
  elif df_est['estimateScaleName'][i] == 'Billions':
    df_est['dataItemValue'][i] = df_est['dataItemValue'][i] * 1_000_000_000
df_est.drop(['estimateScaleName'], axis=1, inplace=True)

#39414143 예상치 데이터가 -e^11이 나옴에 Outlier라고 판단하여 제거
df_est = df_est[df_est['companyId'] != 39414143]
df_est

,companyId,companyName,fiscalQuarter,fiscalYear,periodEndDate,advanceDate,effectiveDate,toDate,periodTypeName,dataItemId,dataItemName,ISOCode,dataItemValue,estimateScaleName
0,24153,Vericel Corporation,0,0,NaT,NaT,2023-09-22 11:08:51,2079-06-06,Non-Periodic,100160,Consensus Recommendation,None,1.600000000,Absolute
1,24153,Vericel Corporation,4,2023,2023-12-31,NaT,2023-09-20 22:05:43,2079-06-06,Annual,100191,EBITDA - # of Estimates,None,4.000000000,Absolute
2,24153,Vericel Corporation,4,2022,2022-12-31,2023-02-23 13:03:00,2023-02-23 13:40:33,2079-06-06,Annual,100191,EBITDA - # of Estimates,None,3.000000000,Absolute
3,24153,Vericel Corporation,4,2021,2021-12-31,2022-02-24 12:55:00,2022-02-24 00:00:00,2079-06-06,Annual,100191,EBITDA - # of Estimates,None,1.000000000,Absolute
4,24153,Vericel Corporation,4,2024,2024-12-31,NaT,2023-09-11 10:03:46,2079-06-06,Annual,100191,EBITDA - # of Estimates,None,4.000000000,Absolute
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92979,1832961004,Cutia Therapeutics,0,0,NaT,NaT,2023-09-04 23:49:32,2079-06-06,Non-Periodic,100165,Target Price - # of Estimates,None,3.000000000,Absolute
92980,1832961004,Cutia Therapeutics,0,0,NaT,NaT,2023-09-04 23:49:32,2079-06-06,Non-Periodic,100163,Target Price Consensus High,CNY,35.229000000,Absolute
92981,1832961004,Cutia Therapeutics,0,0,NaT,NaT,2023-09-04 23:49:32,2079-06-06,Non-Periodic,100164,Target Price Consensus Low,CNY,30.254000000,Absolute
92982,1832961004,Cutia Therapeutics,0,0,NaT,NaT,2023-09-04 23:49:32,2079-06-06,Non-Periodic,100161,Target Price Consensus Mean,CNY,32.658000000,Absolute


In [7]:
#환율 데이터 불러오기 + 21년 ~ 22년까지 Currency별로 데이터프레임 정리
import pyarrow.parquet as pq
import pandas as pd
df_exchange = pd.read_parquet('./exchange')

df_ex = df_exchange.copy()
df_ex.sort_values('priceDate', ascending=True, inplace=True)
df_ex.set_index('priceDate', drop=True, inplace=True)

code = list(df_ex['ISOCode'].unique())
fx = pd.DataFrame()

for c in code:
  ex = df_ex[(df_ex['ISOCode'] == c)].copy()
  ex.rename(columns={'priceClose' : c}, inplace=True)
  ex.drop(['ISOCode'], axis=1, inplace=True)
  d = ex.loc['2021-09-22':'2023-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  fx = pd.concat([fx, d], axis=1)

,ISOCode,priceDate,priceClose
0,PTE,2021-03-20,178.318715000
1,ROL,2021-03-20,25789.000000000
2,SDD,2021-03-20,203.090000000
3,SIT,2021-03-20,213.147798000
4,SKK,2021-03-20,26.795571000
...,...,...,...
494831,BZD,2018-08-17,2.015300000
494832,BZD,2018-08-18,2.015300000
494833,BZD,2018-08-19,2.015300000
494834,BZD,2018-08-20,2.015300000


In [23]:
fx

,GBP,MAD,RUB,NZD,ALL,MVR,SBD,PTE,SZL,LAK,...,DEM,VUV,KWD,KRW,HNL,RON,WST,KPW,BYN,MRU
priceDate,,,,,,,,,,,,,,,,,,,,,
2021-09-22,0.73207,8.9810,72.7162,1.4236,103.60,15.67,8.0580,178.318715,14.7152,9827.8,...,1.739613,111.57,0.30095,1182.65,24.0980,4.2173,2.5589,900.0,2.500,36.04
2021-09-23,0.72812,8.9991,72.8376,1.4135,103.50,15.67,8.0515,178.318715,14.7113,9824.2,...,1.739613,111.57,0.30132,1173.34,24.0980,4.2177,2.5589,900.0,2.500,36.04
2021-09-24,0.73154,8.9999,72.8688,1.4283,103.30,15.42,8.0515,178.318715,14.9820,9824.2,...,1.739613,111.01,0.30096,1181.27,24.1040,4.2273,2.5589,900.0,2.495,36.04
2021-09-25,0.73154,8.9999,72.8688,1.4283,103.30,15.42,8.0515,178.318715,14.9820,9824.2,...,1.739613,111.01,0.30096,1181.27,24.1040,4.2273,2.5589,900.0,2.495,36.04
2021-09-26,0.73154,8.9999,72.8688,1.4283,103.30,15.42,8.0515,178.318715,14.9820,9824.2,...,1.739613,111.01,0.30096,1181.27,24.1040,4.2273,2.5589,900.0,2.495,36.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-18,0.80685,10.2806,96.5002,1.6921,100.20,15.43,8.4034,178.318715,18.9888,19903.5,...,1.739613,121.03,0.30889,1324.24,24.6425,4.6493,2.7824,900.0,2.525,38.08
2023-09-19,0.80721,10.2614,96.7500,1.6869,99.57,15.43,8.4034,178.318715,18.9410,19922.5,...,1.739613,120.99,0.30874,1327.74,24.6420,4.6522,2.7337,900.0,2.525,38.08
2023-09-20,0.80683,10.2333,96.4712,1.6729,99.71,15.46,8.4175,178.318715,18.7688,19960.5,...,1.739613,120.77,0.30856,1325.74,24.6440,4.6377,2.6831,900.0,2.525,38.02


In [28]:
#Estimates 데이터 항목 리스트
estimates_unique = df_est.groupby(['dataItemId','dataItemName', 'periodTypeName']).size().reset_index(name='freq')
estimates_unique.drop('freq', axis=1, inplace=True)

#예상치 데이터 중에 Non-periodic을 제외하고 Annual 데이터만 다룰거라 분류
annual = estimates_unique[estimates_unique['periodTypeName'] == 'Annual']
annual_item = list(annual['dataItemName'])
annual_item

['EPS Normalized Consensus Mean',
 'EPS Normalized Consensus Median',
 'EPS Normalized Consensus High',
 'EPS Normalized Consensus Low',
 'EPS Normalized - # of Estimates',
 'EPS Normalized - Standard Deviation',
 'EPS Normalized Actual',
 'Revenue Consensus Mean',
 'Revenue Consensus Median',
 'Revenue Consensus High',
 'Revenue Consensus Low',
 'Revenue - # of Estimates',
 'Revenue - Standard Deviation',
 'Revenue Actual',
 'EBITDA Consensus Mean',
 'EBITDA Consensus Median',
 'EBITDA Consensus High',
 'EBITDA Consensus Low',
 'EBITDA - # of Estimates',
 'EBITDA - Standard Deviation',
 'EBITDA Actual',
 'Net Income Normalized Consensus Mean',
 'Net Income Normalized Consensus Median',
 'Net Income Normalized Consensus High',
 'Net Income Normalized Consensus Low',
 'Net Income Normalized - # of Estimates',
 'Net Income Normalized - Standard Deviation',
 'Net Income Normalized Actual']

In [85]:
#마켓 데이터로부터 기업의 주가를 시계열로 데이터로 정리

df_price = df_market.copy()
df_price['price'] = df_price['priceClose'] * df_price['adjustmentFactor']
df_price['price'] = df_price['price'].apply(lambda x: round(x, 4))
df_price.drop(['priceClose', 'adjustmentFactor'], axis=1, inplace=True)

df_price['companyId'] = list(map(int, df_price['companyId']))
unique_id = list(df_price['companyId'].unique())
market = pd.DataFrame()

#팩터 모델링을 위한 Training의 기간은 21년 9월 22일부터 22년 9월 22일까지
for id in unique_id:
  ex = df_price[(df_price['companyId'] == id)].copy()
  ex.rename(columns={'price' : id}, inplace=True)
  ex.drop(['companyId'], axis=1, inplace=True)
  ex.sort_values(by='pricingDate', inplace=True)
  ex.set_index('pricingDate', inplace=True, drop=True)
  d = ex.loc['2021-09-22':'2022-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  market = pd.concat([market, d], axis=1)

market2 = market.copy()
mark_date = market2.isnull().sum(axis=1).index
mark_col = market2.isnull().sum(axis=0).index

#한 날짜에 850개 초과의 기업이 주가가 없다면 해당 날짜 제거 (공휴일이라 판단)
for i in mark_date:
  if market2.isnull().sum(axis=1)[i] > 850:
    market2.drop(i, inplace=True)

#한 기업이 40일 이상의 주가가 없으면 제거 (결측치가 너무 많으면 평균이나 선형보간법으로 결측치를 채우는게 무리가 있다고 판단)
for i in mark_col:
  if market2.isnull().sum(axis=0)[i] > 40:
    market2.drop(i, axis=1, inplace=True)

#선형 보간법으로 우선 채우고 연속된 결측치의 경우에는 전 값으로 결측치를 처리
market2.interpolate(method='linear', inplace=True)
market2.fillna(method='bfill', inplace=True)
market2.drop(39414143, axis=1, inplace=True)

In [91]:
market2

,24153,24246,24366,24572,24816,25192,25594,25605,25622,26241,...,713710944,713821465,716991875,1671935602,1673028342,1676815758,1678200093,1679217092,1679642292,1685354464
pricingDate,,,,,,,,,,,,,,,,,,,,,
2021-09-22,48.71,4.3600,12.60,30.81,213.64,1.41,15.39,79.03,2.45,3.05,...,12.650,48.60,14.00,52.45,12.30,24.74,12.40,12.00,9.99,15.650
2021-09-23,50.06,4.4600,12.74,31.91,215.05,1.43,15.37,79.30,2.64,3.05,...,12.500,48.60,13.82,52.45,13.58,24.74,12.40,12.00,9.99,15.650
2021-09-24,49.20,4.7400,12.59,32.08,213.61,1.41,14.92,78.60,2.57,3.04,...,12.700,47.55,13.52,52.40,13.90,24.74,12.40,12.00,9.99,15.650
2021-09-27,49.16,4.7300,12.36,32.08,213.11,1.40,15.38,78.62,2.67,3.10,...,12.600,50.65,13.31,51.30,13.94,24.74,12.40,12.00,9.99,15.650
2021-09-28,48.15,4.7900,11.85,32.13,212.27,1.39,14.73,77.18,2.47,3.00,...,11.800,49.60,13.77,51.10,15.77,24.74,12.40,12.00,9.99,15.650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16,26.04,0.3400,4.67,23.37,231.14,1.95,13.15,88.21,1.29,5.83,...,3.700,14.10,1.21,38.55,13.88,30.82,3.26,2.40,8.90,3.135
2022-09-19,25.92,0.3485,4.62,22.90,230.76,1.61,13.58,87.70,1.22,5.70,...,3.625,12.86,1.17,37.70,14.00,31.50,3.15,2.30,8.51,3.000
2022-09-20,25.54,0.3350,4.67,23.26,227.71,1.55,13.37,86.96,1.21,5.61,...,3.500,13.02,1.14,38.75,14.55,31.96,3.15,2.11,8.28,3.010


In [114]:
#Estimates 데이터 중에 ISOCode가 없는 데이터들이 존재하는데 이를 제거

for a in annual_item:
  d = df_est[(df_est['dataItemName'] == a)]['ISOCode'].isnull().sum()
  if d > 0:
    annual_item.remove(a)

len(annual_item)

24

In [119]:
#한 예상치 데이터에 대해 크기 순으로 정렬하여 상위 50%, 하위 50%를 기준으로 롱숏 포트폴리오를 구성 후 수익률 산출
#예상치 데이터는 연간으로 나타나기 때문에 21년을 데이터를 바탕으로 21년 9월 22일 ~ 21년 12월 31일까지의 포트폴리오를 구성
#22년 데이터에 대해서는 22년 1월 1일~22년 9월 22일까지의 포트폴리오를 구성

def calculate(item, year):
  market3 = market2.copy()
  #아이템이랑 연도 받아서 데이터프레임 생성
  example_feature = df_est[(df_est['dataItemName'] == item)  & (df_est['fiscalYear'] == year)]
  example = example_feature[['companyId', 'dataItemValue', 'ISOCode']].copy()

  #같은 시점에 똑같은 기업이 똑같은 예상치 데이터에 대해 다른 값을 가지고 있다면 평균 내고 하나의 값으로 변환
  #ex) 21년에 A 기업의 EBITDA Estimate이 다른 값으로 2~3개가 있음 -> 평균 내서 하나만 남기기

  example2 = example.copy()
  for id in list(map(int, example2['companyId'])):
    num = len(example2[example2['companyId']==id]['dataItemValue'].values)
    if num > 1:
      idx = list(example2[example2['companyId']==id]['dataItemValue'].index)
      avg = []
      for i in idx:
        a = example2.loc[i]['dataItemValue']
        avg.append(a)
      average = sum(avg) / len(avg)
      example2.loc[idx[0]]['dataItemValue'] = average
      example2.drop([i for i in idx[0:]], inplace=True)
  example2.reset_index(drop=True, inplace=True)

  #예상치 데이터가 어느 나라의 기업이냐에 따라 명시되어 있는 Currency가 다름 (ISOCode를 통해 확인 가능)
  #21년 예상치 데이터를 활용하여 21년 9월 22일부터 ~ 21년 말까지의 포트폴리오를 구성하기 때문에,
  #달러로 표시되어 있는게 아니라면 21년 9월 22일의 환율을 기준으로 값을 달러로 변경
  #22년은 1월 1일 환율을 기준으로 값을 달러로 변경

  if year == 2021:
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2021-09-22", example2['ISOCode'][i]])
  else:
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-01-01", example2['ISOCode'][i]])

  example2.drop('ISOCode', axis=1, inplace=True)
  example2.sort_values(by='dataItemValue', ascending=False, inplace=True)

  #데이터가 있는 기업들의 주가를 불러옴
  comp_id = list(map(int, example2['companyId']))
  market3 = market3.reindex(columns=comp_id)
  market3 = market3.dropna(axis=1)

  #년도에 따라서 포트폴리오를 구성하는 기간이 다르기 때문에 그 구간에서의 주가를 가져옴
  if year == 2021:
    price_y = market3.loc['2021-09-22':'2021-12-31'].copy()
  elif year == 2022:
    price_y = market3.loc['2022-01-01':'2022-09-22'].copy()

  for col in price_y.columns:
    price_y[col] = price_y[col].apply(lambda x: float(x))

  #주가를 수익률로 변환 후에 크기 순으로 정렬되어 있기 때문에 상위 50퍼는 롱,
  #하위 50퍼는 숏을 갖는 포트폴리오의 수익률을 새로운 칼럼으로 생성
  price_y = price_y.pct_change(1).dropna()
  price_y = price_y + 1
  cum_ret = price_y.cumprod() - 1
  half = int(len(cum_ret.columns) / 2)
  cum_ret[item] = (cum_ret.iloc[:, 0:half+1].sum(axis=1) - cum_ret.iloc[:, half+1:].sum(axis=1)) / len(cum_ret.columns)
  return cum_ret[item]

In [120]:
import pyarrow.parquet as pq
import pandas as pd
import statsmodels.api as sm

#21년 9월 22일부터 ~ 22년 9월 22일까지 각 예상치 데이터에 대한 롱숏 포트폴리오를 구성 후 일별 수익률을 계산

pd.options.mode.chained_assignment = None
year = [2022]
multi_agg = pd.DataFrame()
for item in annual_item:
  pf = calculate(item, 2021)
  for y in year:
    temp = calculate(item, y)
    pf = pd.concat([pf, temp], axis=0)
  multi_agg = pd.concat([multi_agg, pf], axis=1)
multi_agg

,EPS Normalized Consensus Mean,EPS Normalized Consensus Median,EPS Normalized Consensus High,EPS Normalized Consensus Low,EPS Normalized - Standard Deviation,EPS Normalized Actual,Revenue Consensus Mean,Revenue Consensus Median,Revenue Consensus High,Revenue Consensus Low,...,EBITDA Consensus High,EBITDA Consensus Low,EBITDA - Standard Deviation,EBITDA Actual,Net Income Normalized Consensus Mean,Net Income Normalized Consensus Median,Net Income Normalized Consensus High,Net Income Normalized Consensus Low,Net Income Normalized - Standard Deviation,Net Income Normalized Actual
2021-09-23,-0.004701,-0.004771,-0.004721,-0.004297,-0.000739,-0.005174,-0.003108,-0.002781,-0.003040,-0.002054,...,-0.002710,-0.001205,-0.002342,-0.001747,-0.001087,-0.001066,-0.000629,0.000324,-0.000804,-0.000915
2021-09-24,-0.001323,-0.001103,-0.001613,-0.000902,-0.002937,-0.001270,-0.004116,-0.003031,-0.005122,-0.001834,...,0.004578,0.006228,-0.005901,0.005563,0.004890,0.004873,0.005289,0.006436,-0.005506,0.005187
2021-09-27,-0.006943,-0.006903,-0.006549,-0.007323,0.002057,-0.006606,-0.005960,-0.005526,-0.007163,-0.004359,...,-0.002567,-0.000271,-0.005408,-0.001240,-0.003108,-0.002985,-0.003290,-0.001645,-0.003883,-0.002615
2021-09-28,-0.004584,-0.004594,-0.003848,-0.005199,-0.000971,-0.004152,-0.006769,-0.005673,-0.007739,-0.005198,...,0.000785,0.003108,-0.006061,0.002626,-0.000294,-0.000134,-0.000538,0.001646,-0.006169,0.000855
2021-09-29,0.003032,0.003534,0.004424,0.002496,-0.005270,0.003730,-0.004299,-0.002916,-0.005884,-0.003127,...,0.007669,0.009560,-0.004670,0.008843,0.005762,0.005965,0.005445,0.007999,-0.008119,0.007328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16,-0.301169,-0.301137,-0.353245,-0.301628,0.074033,-0.336133,-0.019429,-0.016451,-0.282335,-0.028608,...,0.272353,0.265134,-0.222841,0.262248,0.383789,0.381251,0.392537,0.343299,-0.037702,0.388241
2022-09-19,-0.311909,-0.311536,-0.358813,-0.312026,0.076864,-0.346577,-0.012026,-0.008624,-0.282293,-0.020612,...,0.280588,0.274008,-0.229757,0.271145,0.382078,0.379638,0.390222,0.342743,-0.027760,0.386302
2022-09-20,-0.307283,-0.307173,-0.354250,-0.307579,0.078930,-0.344167,-0.013907,-0.010551,-0.277606,-0.023265,...,0.271711,0.265059,-0.228264,0.266042,0.370341,0.367604,0.378358,0.331615,-0.021794,0.374144
2022-09-21,-0.295742,-0.296113,-0.345696,-0.296206,0.072763,-0.336260,-0.009556,-0.007049,-0.266439,-0.017630,...,0.274024,0.267276,-0.233469,0.270362,0.368254,0.365935,0.375897,0.332337,-0.025587,0.371742


In [ ]:
#데이터프레임 생성 과정에서 결측치 존재하면 칼럼 제거
port = multi_agg.copy()
columns_with_missing_values = list(port.columns[port.isnull().any()])
port.drop(columns_with_missing_values, axis=1, inplace=True)

In [123]:
#주가 데이터로부터 수익률을 구하고 첫쨰날을 제거
market4 = market2.copy()
market4 = market4.pct_change(1)
market4.drop('2021-09-22', inplace=True)

#주가 수익률과 포트폴리오 수익률을 합쳐주며 특정 날짜에 Null 값이 존재하면 드랍
estimates_final = pd.concat([market4, port], axis=1)
a = estimates_final.index[estimates_final.isnull().any(axis=1)]
estimates_final.drop(a, inplace=True)
estimates_final

In [134]:
port.corr()

,EPS Normalized Consensus Mean,EPS Normalized Consensus Median,EPS Normalized Consensus High,EPS Normalized Consensus Low,EPS Normalized - Standard Deviation,EPS Normalized Actual,Revenue Consensus Mean,Revenue Consensus Median,Revenue Consensus High,Revenue Consensus Low,...,EBITDA Consensus High,EBITDA Consensus Low,EBITDA - Standard Deviation,EBITDA Actual,Net Income Normalized Consensus Mean,Net Income Normalized Consensus Median,Net Income Normalized Consensus High,Net Income Normalized Consensus Low,Net Income Normalized - Standard Deviation,Net Income Normalized Actual
EPS Normalized Consensus Mean,1.000000,0.999987,0.997177,0.999912,-0.552225,0.998356,-0.237366,-0.237752,-0.161825,-0.234251,...,-0.936695,-0.942287,0.960721,-0.933604,-0.950236,-0.951623,-0.951055,-0.941171,0.229245,-0.951126
EPS Normalized Consensus Median,0.999987,1.000000,0.997177,0.999913,-0.553031,0.998372,-0.237097,-0.237484,-0.161522,-0.233982,...,-0.936466,-0.942104,0.960742,-0.933435,-0.950070,-0.951456,-0.950860,-0.941110,0.228236,-0.950940
EPS Normalized Consensus High,0.997177,0.997177,1.000000,0.997150,-0.575072,0.999528,-0.247989,-0.248361,-0.171303,-0.244886,...,-0.946457,-0.952652,0.969705,-0.944405,-0.955768,-0.957081,-0.956458,-0.949053,0.205600,-0.956509
EPS Normalized Consensus Low,0.999912,0.999913,0.997150,1.000000,-0.551683,0.998435,-0.234709,-0.235102,-0.159020,-0.231597,...,-0.937526,-0.943014,0.961564,-0.934101,-0.950584,-0.951967,-0.951561,-0.941481,0.227885,-0.951437
EPS Normalized - Standard Deviation,-0.552225,-0.553031,-0.575072,-0.551683,1.000000,-0.572429,-0.049408,-0.049551,-0.110478,-0.051190,...,0.446792,0.473069,-0.649800,0.457091,0.417944,0.420076,0.421938,0.428422,0.622155,0.419469
EPS Normalized Actual,0.998356,0.998372,0.999528,0.998435,-0.572429,1.000000,-0.237365,-0.237740,-0.160736,-0.234258,...,-0.941844,-0.948150,0.969624,-0.939228,-0.951813,-0.953218,-0.952737,-0.944273,0.204172,-0.952563
Revenue Consensus Mean,-0.237366,-0.237097,-0.247989,-0.234709,-0.049408,-0.237365,1.000000,0.999999,0.996634,0.999994,...,0.372612,0.362058,-0.160087,0.392287,0.376489,0.373524,0.356331,0.388891,-0.413317,0.379176
Revenue Consensus Median,-0.237752,-0.237484,-0.248361,-0.235102,-0.049551,-0.237740,0.999999,1.000000,0.996608,0.999993,...,0.373054,0.362485,-0.160417,0.392699,0.376933,0.373969,0.356779,0.389330,-0.413811,0.379619
Revenue Consensus High,-0.161825,-0.161522,-0.171303,-0.159020,-0.110478,-0.160736,0.996634,0.996608,1.000000,0.996867,...,0.300395,0.288782,-0.080544,0.320287,0.305588,0.302460,0.284938,0.317874,-0.421054,0.308322
Revenue Consensus Low,-0.234251,-0.233982,-0.244886,-0.231597,-0.051190,-0.234258,0.999994,0.999993,0.996867,1.000000,...,0.369821,0.359247,-0.157139,0.389516,0.373621,0.370652,0.353446,0.386084,-0.412721,0.376305


In [136]:
#Total은 바이오 섹터에 속하는 개별 기업의 개수
total = len(estimates_final.columns[:-24])
#Port_col은 각 포트폴리오의 이름을 의미
port_col = list(port.columns)

#개별 기업의 수익률을 y로 두고, 각 데이터에 대한 포트폴리오를 X로 두고 1158개에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in port_col:
  cnt = 0
  for col in estimates_final.columns[:-24]:
    res = sm.OLS(endog=estimates_final[col], exog=sm.add_constant(estimates_final[p])).fit()-
    if res.pvalues[1] <= 0.05:
      cnt += 1
  print(p, cnt)

EPS Normalized Consensus Mean 50
EPS Normalized Consensus Median 50
EPS Normalized Consensus High 53
EPS Normalized Consensus Low 50
EPS Normalized - Standard Deviation 53
EPS Normalized Actual 53
Revenue Consensus Mean 25
Revenue Consensus Median 25
Revenue Consensus High 23
Revenue Consensus Low 25
Revenue - Standard Deviation 25
Revenue Actual 25
EBITDA Consensus Mean 41
EBITDA Consensus Median 41
EBITDA Consensus High 39
EBITDA Consensus Low 41
EBITDA - Standard Deviation 44
EBITDA Actual 38
Net Income Normalized Consensus Mean 41
Net Income Normalized Consensus Median 41
Net Income Normalized Consensus High 41
Net Income Normalized Consensus Low 39
Net Income Normalized - Standard Deviation 47
Net Income Normalized Actual 40
